<a href="https://colab.research.google.com/github/atilatech/atlas-service/blob/master/notebooks/deploy_whisper_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers pytube
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

In [15]:
!pip install git+https://github.com/openai/whisper.git -q
!apt install ffmpeg # https://stackoverflow.com/questions/51856340/how-to-install-package-ffmpeg-in-google-colab

# optional install pytorch so you can use a gpu for faster transcription
# command below is for Linux. See instructions for mac and windows: https://pytorch.org/get-started/locally/
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.11-0ubuntu0.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cpu


In [21]:
import torch

import pytube as pt
import whisper

MODEL_NAME = "tiny.en" #this always needs to stay in line 8 :D sorry for the hackiness
lang = "en"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("using device: ", device)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'whisper will use: {device}')

whisper_model = whisper.load_model(MODEL_NAME).to(device)



using device:  cpu
whisper will use: cpu


In [26]:

import time
def yt_transcribe(yt_url):


    decode_options = {
        # Set language to None to support multilingual, 
        # but it will take longer to process while it detects the language.
        # Realized this by running in verbose mode and seeing how much time
        # was spent on the decoding language step
        "language":"en",
        "verbose": True
      }
    yt = pt.YouTube(yt_url)
    stream = yt.streams.filter(only_audio=True)[0]
    path_to_audio = f"{yt.video_id}.mp3"
    stream.download(filename=path_to_audio)

    # with open(path_to_audio, "rb") as i:
    #     audio_binary = i.read()

    t0 = time.time()
    transcript = whisper_model.transcribe(path_to_audio, **decode_options)
    t1 = time.time()
    
    total = t1-t0
    print(f'Finished transcription in {total} seconds')
    return transcript

In [27]:
video_url="https://www.youtube.com/watch?v=aNxigRg1yEQ"
transcription = yt_transcribe(video_url)

/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:03.920]  Because if I just think, well tomorrow I'm just gonna coast and eat Twinkies and watch TV.
[00:03.920 --> 00:06.400]  Oh, hello, sadness, my old friend.
[00:06.400 --> 00:07.600]  Hello, depression.
[00:07.600 --> 00:09.920]  Because when you're not doing anything you feel like shit.
[00:09.920 --> 00:12.000]  And that's just a part of being a human being.
[00:12.000 --> 00:14.800]  And we can pretend that we're something other than what we really are.
[00:14.800 --> 00:18.800]  And we can pretend, nah, me man, I'm just cool, just chilling, doing nothing.
[00:18.800 --> 00:20.800]  Bullshit. You're a fucking human.
[00:20.800 --> 00:26.480]  You're a human being. You evolve from the fucking hundreds of thousands of years of hunters and gatherers
[00:26.480 --> 00:31.440]  and people who are struggling. Human reward systems are carved deeply into your DNA.
[00:31.440 --> 00:36.560]  And if you don't respect that, if you don't respect the mechanism of happiness

In [ ]:
transcription

# Create a custom handler

In [30]:
from typing import  Dict
from transformers.pipelines.audio_utils import ffmpeg_read
import whisper
import torch
import pytube
import time


class EndpointHandler():
    def __init__(self, path=""):
        # load the model
        MODEL_NAME = "tiny.en"
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f'whisper will use: {device}')
        
        t0 = time.time()
        self.model = whisper.load_model(MODEL_NAME).to(device)
        t1 = time.time()
        
        total = t1-t0
        print(f'Finished loading model in {total} seconds')


    def __call__(self, data: Dict[str, bytes]) -> Dict[str, str]:
        """
        Args:
            data (:obj:):
                includes the URL to video for transcription
        Return:
            A :obj:`dict`:. transcribed dict
        """
        # process input
        print('data', data)
        video_url = data.pop("inputs", data)
        decode_options = {
            # Set language to None to support multilingual, 
            # but it will take longer to process while it detects the language.
            # Realized this by running in verbose mode and seeing how much time
            # was spent on the decoding language step
            "language":"en",
            "verbose": True
        }
        yt = pytube.YouTube(video_url)
        stream = yt.streams.filter(only_audio=True)[0]
        path_to_audio = f"{yt.video_id}.mp3"
        stream.download(filename=path_to_audio)
        t0 = time.time()
        transcript = self.model.transcribe(path_to_audio, **decode_options)
        t1 = time.time()
        
        total = t1-t0
        print(f'Finished transcription in {total} seconds')
        

        # postprocess the prediction
        return {"transcript": transcript}


In [31]:
# Use Handler

my_handler = EndpointHandler(path=".")

# prepare sample payload
payload = {"inputs": "https://www.youtube.com/watch?v=aNxigRg1yEQ"}
holiday_payload = {"inputs": "Today is a though day", "date": "2022-07-04"}

# test the handler
payload_pred=my_handler(payload)

whisper will use: cpu
Finished loading model in 0.3354830741882324 seconds
data {'inputs': 'https://www.youtube.com/watch?v=aNxigRg1yEQ'}


/usr/local/lib/python3.8/dist-packages/whisper/transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


[00:00.000 --> 00:03.920]  Because if I just think, well tomorrow I'm just gonna coast and eat Twinkies and watch TV.
[00:03.920 --> 00:06.400]  Oh, hello, sadness, my old friend.
[00:06.400 --> 00:07.600]  Hello, depression.
[00:07.600 --> 00:09.920]  Because when you're not doing anything you feel like shit.
[00:09.920 --> 00:12.000]  And that's just a part of being a human being.
[00:12.000 --> 00:14.800]  And we can pretend that we're something other than what we really are.
[00:14.800 --> 00:18.800]  And we can pretend, nah, me man, I'm just cool, just chilling, doing nothing.
[00:18.800 --> 00:20.800]  Bullshit. You're a fucking human.
[00:20.800 --> 00:26.480]  You're a human being. You evolve from the fucking hundreds of thousands of years of hunters and gatherers
[00:26.480 --> 00:31.440]  and people who are struggling. Human reward systems are carved deeply into your DNA.
[00:31.440 --> 00:36.560]  And if you don't respect that, if you don't respect the mechanism of happiness

In [32]:
payload_pred

{'transcript': {'text': " Because if I just think, well tomorrow I'm just gonna coast and eat Twinkies and watch TV. Oh, hello, sadness, my old friend. Hello, depression. Because when you're not doing anything you feel like shit. And that's just a part of being a human being. And we can pretend that we're something other than what we really are. And we can pretend, nah, me man, I'm just cool, just chilling, doing nothing. Bullshit. You're a fucking human. You're a human being. You evolve from the fucking hundreds of thousands of years of hunters and gatherers and people who are struggling. Human reward systems are carved deeply into your DNA. And if you don't respect that, if you don't respect the mechanism of happiness and fulfillment and what you really need to do in order to feel satisfied in life, comradery, love, family, friendship, struggle, testing yourself, learning, all those things are imperative. They're all a giant part of being a person.",
  'segments': [{'id': 0,
    'see